Visualization babysteps. 
We start with the code of Ch 3 of 'Data Science from scratch' by Joel Grus
But we'll work with real gdp data

In [13]:
# all imports at the top
from matplotlib import pyplot as plt
import ipywidgets as widgets
import pandas as pd
import requests
import zipfile
from io import BytesIO
from pathlib import Path


In [ ]:
# helper functions
def download_extract_gdp_data(url = 'https://api.worldbank.org/v2/en/indicator/NY.GDP.MKTP.PP.CD?downloadformat=csv', destination_file : Path = Path('../data/gdp_ppp.csv'))-> None:
    """
    Download and extract GDP data from the World Bank API.
    
    Parameters:
    url (str): The URL to download the data from.
    destination_file (Path): The path to the destination file where it saves the extracted file. Defaults to '../data/gdp_ppp.csv'.
    
    Returns:
    None
    """
    print(f"Downloading data from the World Bank API {url} to {destination_file} ...")
    # download the csv file from the World Bank API
    response = requests.get(url)
    # unzip the contents, find the csv file that starts with API, and save it to the data folder
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        for filename in z.namelist():
            if filename.startswith('API_') and filename.endswith('.csv'):
                with open(destination_file, 'wb') as f:
                    f.write(z.read(filename))
                # save the csv file to the data folder as gdp_ppp.csv
                print(f"Downloaded {filename} to {destination_file}")
                break
    

In [15]:
# check if data folder has the gdp-ppp csv file
# if not, download it from the World Bank API

data_folder = Path("../data")
data_folder.mkdir(exist_ok=True)
csv_file = data_folder.joinpath("gdp_ppp.csv")
#TODO: look at the lates data on the World Bank API and compare its metadata with the one in the csv file, 
# if they are different, download the new csv file
if not csv_file.exists():
    download_extract_gdp_data(destination_file=csv_file) 
# read the csv file into a pandas dataframe
df = pd.read_csv(csv_file, skiprows=4)

Downloaded API_NY.GDP.MKTP.PP.CD_DS2_en_csv_v2_13534.csv to ../data/gdp_ppp.csv
